In [2]:
import numpy as np

In [3]:
import keras

In [4]:
import tensorflow as tf

In [227]:
from tensorflow.keras.layers import MultiHeadAttention

In [228]:
ma=MultiHeadAttention(num_heads=8,key_dim=512)

In [229]:
x=np.random.random((150,10,512))
y=np.random.random((150,15,512))

In [344]:
mask=True

In [345]:
out,attn=ma(y,x,return_attention_scores=True,use_causal_mask=mask)

In [346]:
out.shape

TensorShape([150, 15, 512])

In [347]:
attn.shape

TensorShape([150, 8, 15, 10])

In [370]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:,tf.newaxis,tf.newaxis,:]
    
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    print("before:",mask)
    return mask*-1e9
    
def create_masks(inputs, target):
    enc_padding_mask = create_padding_mask(inputs)
    dec_padding_mask = create_padding_mask(target)
    look_ahead_mask = create_look_ahead_mask(tf.shape(target)[1])
    print("after:",look_ahead_mask)
    print(look_ahead_mask.shape)
    look_ahead_mask = tf.maximum(dec_padding_mask, look_ahead_mask)
    return enc_padding_mask, look_ahead_mask, dec_padding_mask

In [371]:
m=np.arange(9).reshape(3,3)

In [372]:
n=np.arange(12).reshape(3,4)

In [373]:
m.shape,n.shape

((3, 3), (3, 4))

In [374]:
m=np.array(m)
n=np.array(n)

In [376]:
l1,l2,l3=create_masks(m,n)

before: tf.Tensor(
[[0. 1. 1. 1.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]], shape=(4, 4), dtype=float32)
after: tf.Tensor(
[[-0.e+00 -1.e+09 -1.e+09 -1.e+09]
 [-0.e+00 -0.e+00 -1.e+09 -1.e+09]
 [-0.e+00 -0.e+00 -0.e+00 -1.e+09]
 [-0.e+00 -0.e+00 -0.e+00 -0.e+00]], shape=(4, 4), dtype=float32)
(4, 4)


In [380]:
tf.maximum(l3, l2)

<tf.Tensor: shape=(3, 1, 4, 4), dtype=float32, numpy=
array([[[[ 1.,  0.,  0.,  0.],
         [ 1., -0.,  0.,  0.],
         [ 1., -0., -0.,  0.],
         [ 1., -0., -0., -0.]]],


       [[[-0.,  0.,  0.,  0.],
         [-0., -0.,  0.,  0.],
         [-0., -0., -0.,  0.],
         [-0., -0., -0., -0.]]],


       [[[-0.,  0.,  0.,  0.],
         [-0., -0.,  0.,  0.],
         [-0., -0., -0.,  0.],
         [-0., -0., -0., -0.]]]], dtype=float32)>

In [306]:
l1.shape,l2.shape,l3.shape

(TensorShape([3, 1, 1, 3]),
 TensorShape([3, 1, 4, 4]),
 TensorShape([3, 1, 1, 4]))

In [308]:
xy=np.random.random((3,8,4,3))

In [339]:
l2

<tf.Tensor: shape=(3, 1, 4, 4), dtype=float32, numpy=
array([[[[1., 1., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 0., 1.],
         [1., 0., 0., 0.]]],


       [[[0., 1., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 0.]]],


       [[[0., 1., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 0.]]]], dtype=float32)>

In [289]:
(xy+l3).shape

TensorShape([3, 8, 4, 4])

In [332]:
a=np.arange(12).reshape(3,1,1,4)

In [333]:
b=np.arange(20,36).reshape(4,4)

In [335]:
c=np.maximum(a,b)

In [336]:
c.shape

(3, 1, 4, 4)